<a href="https://colab.research.google.com/github/preetbhagat7/localrepo/blob/main/Random_F_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report


In [30]:
from google.colab import files
uploaded =files.upload()

Saving training-data.json to training-data (2).json


In [63]:
data = pd.read_json('training-data.json')
print("Initial Data:")
print(data.head(2))
print(data.shape)
print(data.info())

Initial Data:
                             session_id last_event_timestamp  \
0  de478b38-5cb1-40fe-bbbd-1b87e149b8e3  2025-11-06 09:29:28   
1  f82e54ca-9eda-4bd2-b15f-f4310abd240a  2025-11-06 09:23:49   

                                brand  \
0  lifelong-electronics.myshopify.com   
1  lifelong-electronics.myshopify.com   

                                           user_id  device_tier  \
0             4cceb738-df7c-4d16-9152-d48746a878e9          4.0   
1  t4g74gew8hn-c3z8v0ysvs8-oxos8h32xoh-the1uxysct8          4.0   

           device_name  device_recency device_manufacturer  \
0           Galaxy A71             4.0             Samsung   
1  Apple iPhone 15 Pro             5.0               Apple   

                                          user_agent  session_start_time  ...  \
0  Mozilla/5.0 (Linux; Android 10; K) AppleWebKit... 2025-11-06 09:28:14  ...   
1  Mozilla/5.0 (iPhone; CPU iPhone OS 18_5 like M... 2025-11-06 09:19:42  ...   

  is_battery_charging  weather_condi

drop nested json data

In [45]:
cols_to_drop = [
    'session_id', 'user_id', 'user_agent',
    'reference_ids', 'ip_info', 'form_info', 'ip_data', 'form_data'
]
data = data.drop(columns=cols_to_drop, errors='ignore')

In [60]:
print(data)

     last_event_timestamp                               brand  device_tier  \
0              1762421368  lifelong-electronics.myshopify.com          4.0   
1              1762421029  lifelong-electronics.myshopify.com          4.0   
2              1762417973  lifelong-electronics.myshopify.com          4.0   
3              1762419750  lifelong-electronics.myshopify.com          NaN   
4              1762417832  lifelong-electronics.myshopify.com          4.0   
..                    ...                                 ...          ...   
495            1762420231  lifelong-electronics.myshopify.com          4.0   
496            1762420196  lifelong-electronics.myshopify.com          4.0   
497            1762420183  lifelong-electronics.myshopify.com          4.0   
498            1762420536  lifelong-electronics.myshopify.com          NaN   
499            1762420206  lifelong-electronics.myshopify.com          4.0   

                   device_name  device_recency device_manufactu

convert date column into numeric timestamp

In [46]:
datetime_cols = ['last_event_timestamp', 'session_start_time', 'session_end_time']
for col in datetime_cols:
    if col in data.columns:
        data[col] = pd.to_datetime(data[col], errors='coerce')
        data[col] = data[col].astype('int64') // 10**9

In [64]:
print("\nAfter Datetime Conversion:")
print(data.head())
print(data.info())


After Datetime Conversion:
                             session_id last_event_timestamp  \
0  de478b38-5cb1-40fe-bbbd-1b87e149b8e3  2025-11-06 09:29:28   
1  f82e54ca-9eda-4bd2-b15f-f4310abd240a  2025-11-06 09:23:49   
2  67e3f350-c003-4a0c-a21a-76a01d507c4b  2025-11-06 08:32:53   
3  a3887079-8081-4246-a52b-76707c210b1e  2025-11-06 09:02:30   
4  d55e7cb7-78d8-48be-a793-4fb361d903ec  2025-11-06 08:30:32   

                                brand  \
0  lifelong-electronics.myshopify.com   
1  lifelong-electronics.myshopify.com   
2  lifelong-electronics.myshopify.com   
3  lifelong-electronics.myshopify.com   
4  lifelong-electronics.myshopify.com   

                                           user_id  device_tier  \
0             4cceb738-df7c-4d16-9152-d48746a878e9          4.0   
1  t4g74gew8hn-c3z8v0ysvs8-oxos8h32xoh-the1uxysct8          4.0   
2             80a81a1e-f3b8-4a69-b193-fd9051230c98          4.0   
3             aa2c35ea-4ab2-4914-baec-c8f8322120db          NaN   
4    

In [61]:
display(data.describe)

<bound method NDFrame.describe of      last_event_timestamp                               brand  device_tier  \
0              1762421368  lifelong-electronics.myshopify.com          4.0   
1              1762421029  lifelong-electronics.myshopify.com          4.0   
2              1762417973  lifelong-electronics.myshopify.com          4.0   
3              1762419750  lifelong-electronics.myshopify.com          NaN   
4              1762417832  lifelong-electronics.myshopify.com          4.0   
..                    ...                                 ...          ...   
495            1762420231  lifelong-electronics.myshopify.com          4.0   
496            1762420196  lifelong-electronics.myshopify.com          4.0   
497            1762420183  lifelong-electronics.myshopify.com          4.0   
498            1762420536  lifelong-electronics.myshopify.com          NaN   
499            1762420206  lifelong-electronics.myshopify.com          4.0   

                   device_name  device_recency device_manufacturer  \
0                   Galaxy A71             4.0             Samsung   
1          Apple iPhone 15 Pro             5.0               Apple   
2                          V40             4.0                Vivo   
3                                          NaN                       
4    OnePlus Nord CE 3 Lite 5G             4.0             OnePlus   
..                         ...             ...                 ...   
495                 Galaxy M12             4.0             Samsung   
496                    Nexus 5             4.0                 LGE   
497                   realme 5             4.0                Oppo   
498                                        NaN                       
499       Redmi Note 12 Pro 5G             4.0               Redmi   

     session_start_time  session_end_time  visit_count  session_duration_sec  \
0            1762421294        1762421368            1                    74   
1            1762420782        1762421029            2                   247   
2            1762417894        1762417973            1                    79   
3            1762417804        1762419750            4                  1946   
4            1762417802        1762417832            4                    30   
..                  ...               ...          ...                   ...   
495          1762420209        1762420231            1                    22   
496          1762420196        1762420196            1                     0   
497          1762420183        1762420183            1                     0   
498          1762420181        1762420536            2                   355   
499          1762420179        1762420206            7                    27   

     ... os_version                network_provider       ip_address  \
0    ...     12.0.0             Tikona Infinet Ltd.       1.22.83.13   
1    ...       18.5  Hathway IP Over Cable Internet     116.74.71.66   
2    ...     15.0.0                          Airtel  223.231.139.154   
3    ...         10        Gujarat Telelink Pvt Ltd   103.84.198.184   
4    ...     15.0.0             DWAN SUPPORTS P LTD    103.17.110.45   
..   ...        ...                             ...              ...   
495  ...     13.0.0                          Airtel   223.191.58.245   
496  ...      6.0.1                      Google LLC     66.249.68.65   
497  ...     10.0.0             ACTFIBERNET Pvt Ltd    49.206.116.71   
498  ...     18.6.2                                             None   
499  ...     12.0.0                             Jio     49.47.143.62   

    internet_speed battery_level is_battery_charging weather_condition  \
0               4g            86               False                     
1                              0               False                     
2               4g            43                True                     
3               4g           100                True              

remove missing data from target

In [47]:
data = data.dropna(subset=['has_purchase'])


In [65]:
print("\nAfter Handling Missing Values:")
print(data.head())
print(data.shape)


After Handling Missing Values:
                             session_id last_event_timestamp  \
0  de478b38-5cb1-40fe-bbbd-1b87e149b8e3  2025-11-06 09:29:28   
1  f82e54ca-9eda-4bd2-b15f-f4310abd240a  2025-11-06 09:23:49   
2  67e3f350-c003-4a0c-a21a-76a01d507c4b  2025-11-06 08:32:53   
3  a3887079-8081-4246-a52b-76707c210b1e  2025-11-06 09:02:30   
4  d55e7cb7-78d8-48be-a793-4fb361d903ec  2025-11-06 08:30:32   

                                brand  \
0  lifelong-electronics.myshopify.com   
1  lifelong-electronics.myshopify.com   
2  lifelong-electronics.myshopify.com   
3  lifelong-electronics.myshopify.com   
4  lifelong-electronics.myshopify.com   

                                           user_id  device_tier  \
0             4cceb738-df7c-4d16-9152-d48746a878e9          4.0   
1  t4g74gew8hn-c3z8v0ysvs8-oxos8h32xoh-the1uxysct8          4.0   
2             80a81a1e-f3b8-4a69-b193-fd9051230c98          4.0   
3             aa2c35ea-4ab2-4914-baec-c8f8322120db          NaN   
4

In [48]:
X = data.drop('has_purchase', axis=1)
y = data['has_purchase']


In [66]:
print("\nFeatures & Target Shapes:")
print(X.shape, y.shape)


Features & Target Shapes:
(500, 1614) (500,)


convert categorical data to numerical

In [49]:
X = pd.get_dummies(X, drop_first=True)


In [67]:
print("\nAfter Encoding:")
print(X.head())
print(X.shape)


After Encoding:
   last_event_timestamp  device_tier  device_recency  session_start_time  \
0            1762421368          4.0             4.0          1762421294   
1            1762421029          4.0             5.0          1762420782   
2            1762417973          4.0             4.0          1762417894   
3            1762419750          NaN             NaN          1762417804   
4            1762417832          4.0             4.0          1762417802   

   session_end_time  visit_count  session_duration_sec        utm_id  \
0        1762421368            1                    74  1.202386e+17   
1        1762421029            2                   247  1.202373e+17   
2        1762417973            1                    79           NaN   
3        1762419750            4                  1946           NaN   
4        1762417832            4                    30  1.202372e+17   

   is_bounced  is_engaged  ...  primary_language_zh-CN  \
0           0           1  ...     

split data

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
print("\nTrain/Test Shapes:")
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


Train/Test Shapes:
(400, 1614) (100, 1614) (400,) (100,)


train data

In [55]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

predict data

In [54]:
y_pred = model.predict(X_test)

In [57]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        54
           1       1.00      0.98      0.99        46

    accuracy                           0.99       100
   macro avg       0.99      0.99      0.99       100
weighted avg       0.99      0.99      0.99       100



evaluate

In [59]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)*100
print("Accuracy:", accuracy)

Accuracy: 99.0
